In [33]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (123 kB/s

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/FS.csv");
df.show()

+-------+---+------+---------+
|Country|Age|Salary|Purchased|
+-------+---+------+---------+
| France| 44| 72000|       No|
|  Spain| 27| 48000|      Yes|
|Germany| 30| 54000|       No|
|  Spain| 38| 61000|       No|
|Germany| 40|  1000|      Yes|
| France| 35| 58000|      Yes|
|  Spain| 78| 52000|       No|
| France| 48| 79000|      Yes|
|Germany| 50| 83000|       No|
| France| 37| 67000|      Yes|
+-------+---+------+---------+



In [ ]:
categories = df.select('Country').distinct().collect();
Dictinct_Country_Names=[];
for i in categories:
  Dictinct_Country_Names.append(i.asDict()['Country'])

In [ ]:
categories

[Row(Country='Germany'), Row(Country='France'), Row(Country='Spain')]

In [ ]:
Dictinct_Country_Names

['Germany', 'France', 'Spain']

In [ ]:
def One_Hot_Encodinga(item,cate):
  if(item==cate):
    return 1;
  else:
    return 0; 

In [ ]:
one_hota=udf(One_Hot_Encodinga,IntegerType())

In [ ]:
for category in Dictinct_Country_Names:
    new_column_name = 'col2'+'_'+category
    df = df.withColumn(new_column_name,one_hota('Country',lit(category)));

In [ ]:
df.show()

+-------+---+------+---------+------------+-----------+----------+
|Country|Age|Salary|Purchased|col2_Germany|col2_France|col2_Spain|
+-------+---+------+---------+------------+-----------+----------+
| France| 44| 72000|       No|           0|          1|         0|
|  Spain| 27| 48000|      Yes|           0|          0|         1|
|Germany| 30| 54000|       No|           1|          0|         0|
|  Spain| 38| 61000|       No|           0|          0|         1|
|Germany| 40|  1000|      Yes|           1|          0|         0|
| France| 35| 58000|      Yes|           0|          1|         0|
|  Spain| 78| 52000|       No|           0|          0|         1|
| France| 48| 79000|      Yes|           0|          1|         0|
|Germany| 50| 83000|       No|           1|          0|         0|
| France| 37| 67000|      Yes|           0|          1|         0|
+-------+---+------+---------+------------+-----------+----------+

